In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

class SimpleMentalHealthPredictor:
    def __init__(self):
        # Keyword lists for detection
        self.keywords = {
            'suicidal': ['suicide', 'kill myself', 'end it all', 'not worth living', 
                        'better off dead', 'want to die', 'no point', 'can\'t go on'],
            'depression': ['hopeless', 'worthless', 'empty', 'sad', 'depressed', 
                          'tired', 'exhausted', 'meaningless', 'lonely', 'defeated'],
            'anxiety': ['worried', 'anxious', 'panic', 'nervous', 'scared', 'fear', 
                       'overwhelmed', 'racing thoughts', 'restless', 'tense'],
            'stress': ['stressed', 'pressure', 'overwhelmed', 'burned out', 'rushed', 
                      'frantic', 'overloaded', 'deadline', 'hectic', 'chaotic']
        }
        
        # Simple mindfulness habits list
        self.habits = [
            'morning_meditation', 'gratitude_journal', 'nature_walk', 'box_breathing',
            'stretching', 'journaling', 'mindful_eating', 'digital_detox'
        ]

    def count_keywords(self, text, keyword_list):
        """Count how many keywords appear in text"""
        if not text:
            return 0
        text_lower = text.lower()
        count = 0
        for keyword in keyword_list:
            if keyword in text_lower:
                count += 1
        return count

    def predict_condition(self, mood_score, habit_completion_rate, text_input, dass_scores=None):
        """
        Main prediction function - returns condition and resources
        
        Args:
            mood_score (int): 1-10 mood rating
            habit_completion_rate (float): 0-1 (0% to 100%)
            text_input (str): User's text input
            dass_scores (dict): Optional DASS-21 scores
        
        Returns:
            dict: condition, explanation, resources
        """
        
        # Count keywords in text
        suicidal_keywords = self.count_keywords(text_input, self.keywords['suicidal'])
        depression_keywords = self.count_keywords(text_input, self.keywords['depression'])
        anxiety_keywords = self.count_keywords(text_input, self.keywords['anxiety'])
        stress_keywords = self.count_keywords(text_input, self.keywords['stress'])
        
        # Simple rule-based classification
        
        # 1. Check for suicidal ideation first (highest priority)
        if suicidal_keywords > 0 or mood_score <= 2:
            condition = 'suicidal_ideation'
            
        # 2. Check for depression
        elif depression_keywords >= 2 or (mood_score <= 4 and habit_completion_rate <= 0.3):
            condition = 'depression'
            
        # 3. Check for anxiety
        elif anxiety_keywords >= 2 or (text_input and 'panic' in text_input.lower()):
            condition = 'anxiety'
            
        # 4. Check for stress
        elif stress_keywords >= 2 or habit_completion_rate <= 0.4:
            condition = 'stress'
            
        # 5. Healthy state
        elif mood_score >= 7 and habit_completion_rate >= 0.7:
            condition = 'healthy'
            
        # 6. Default to mild concern
        else:
            condition = 'mild_concern'
        
        # Generate simple explanation
        explanation = self._create_explanation(mood_score, habit_completion_rate, 
                                             suicidal_keywords, depression_keywords,
                                             anxiety_keywords, stress_keywords)
        
        # Get resources
        resources = self._get_resources(condition)
        
        return {
            'condition': condition,
            'explanation': explanation,
            'resources': resources,
            'needs_crisis_intervention': condition == 'suicidal_ideation'
        }

    def _create_explanation(self, mood, habits, suicidal, depression, anxiety, stress):
        """Create simple explanation for the prediction"""
        parts = []
        
        if suicidal > 0:
            parts.append("Concerning language detected")
        elif mood <= 3:
            parts.append(f"Low mood score ({mood}/10)")
        elif mood >= 8:
            parts.append(f"Good mood score ({mood}/10)")
            
        if habits <= 0.3:
            parts.append("Low habit completion suggests difficulty with self-care")
        elif habits >= 0.8:
            parts.append("Great job maintaining healthy habits!")
            
        if depression >= 2:
            parts.append("Language suggests depressive symptoms")
        if anxiety >= 2:
            parts.append("Language suggests anxiety symptoms")
        if stress >= 2:
            parts.append("Language suggests stress")
            
        return ". ".join(parts) if parts else "General assessment based on inputs"

    def _get_resources(self, condition):
        """Get appropriate resources based on condition"""
        resources = {
            'suicidal_ideation': {
                'immediate_actions': [
                    "🚨 If in immediate danger, call 911",
                    "📞 National Suicide Prevention Lifeline: 988",
                    "💬 Crisis Text Line: Text HOME to 741741"
                ],
                'coping_strategies': [
                    "Reach out to a trusted person immediately",
                    "Remove harmful objects from your area",
                    "Try box breathing: 4 counts in, hold 4, out 4, hold 4"
                ],
                'professional_help': "Please contact a mental health professional immediately"
            },
            
            'depression': {
                'immediate_actions': [
                    "Take a 5-minute walk outside",
                    "Write down 3 things you're grateful for",
                    "Call or text a friend or family member"
                ],
                'coping_strategies': [
                    "Try to maintain a daily routine",
                    "Do one small pleasant activity today",
                    "Practice deep breathing for 2 minutes"
                ],
                'professional_help': "Consider speaking with a counselor or therapist"
            },
            
            'anxiety': {
                'immediate_actions': [
                    "Try the 5-4-3-2-1 grounding technique",
                    "Practice box breathing for 2 minutes",
                    "Do progressive muscle relaxation"
                ],
                'coping_strategies': [
                    "Challenge worried thoughts - are they realistic?",
                    "Limit caffeine intake",
                    "Take breaks from social media/news"
                ],
                'professional_help': "Consider anxiety management techniques or counseling"
            },
            
            'stress': {
                'immediate_actions': [
                    "Take 5 deep breaths right now",
                    "Step away from stressful tasks for 10 minutes",
                    "Do gentle stretching"
                ],
                'coping_strategies': [
                    "Prioritize your most important tasks",
                    "Take regular breaks throughout the day",
                    "Practice saying 'no' to additional commitments"
                ],
                'professional_help': "Consider stress management counseling if ongoing"
            },
            
            'healthy': {
                'immediate_actions': [
                    "Keep up the great work!",
                    "Continue your healthy habits",
                    "Share your positive energy with others"
                ],
                'coping_strategies': [
                    "Maintain your current routine",
                    "Consider helping others who might be struggling",
                    "Stay connected with supportive people"
                ],
                'professional_help': "Continue regular mental health check-ins"
            },
            
            'mild_concern': {
                'immediate_actions': [
                    "Take a few minutes for mindful breathing",
                    "Go for a short walk or stretch",
                    "Practice gratitude - think of 1 good thing today"
                ],
                'coping_strategies': [
                    "Try to complete 1-2 mindfulness habits today",
                    "Connect with a friend or loved one",
                    "Get adequate sleep (7-8 hours)"
                ],
                'professional_help': "Monitor your symptoms and consider counseling if they persist"
            }
        }
        
        return resources.get(condition, resources['mild_concern'])

# EASY-TO-USE DEMO FUNCTIONS

def quick_assessment(mood, habit_rate, text):
    """Quick assessment function for testing"""
    predictor = SimpleMentalHealthPredictor()
    result = predictor.predict_condition(mood, habit_rate, text)
    
    print(f"🧠 CONDITION: {result['condition'].upper()}")
    print(f"📝 EXPLANATION: {result['explanation']}")
    print(f"🆘 CRISIS INTERVENTION NEEDED: {result['needs_crisis_intervention']}")
    print(f"\n💡 IMMEDIATE ACTIONS:")
    for action in result['resources']['immediate_actions']:
        print(f"   • {action}")
    print(f"\n🔧 COPING STRATEGIES:")
    for strategy in result['resources']['coping_strategies']:
        print(f"   • {strategy}")
    
    return result

def test_all_conditions():
    """Test the model with different scenarios"""
    predictor = SimpleMentalHealthPredictor()
    
    test_cases = [
        {
            'name': 'Crisis Situation',
            'mood': 1,
            'habits': 0.1,
            'text': 'I want to end it all, cant go on anymore'
        },
        {
            'name': 'Depression',
            'mood': 3,
            'habits': 0.2,
            'text': 'feeling hopeless and worthless, everything is meaningless'
        },
        {
            'name': 'Anxiety',
            'mood': 5,
            'habits': 0.6,
            'text': 'really anxious about everything, heart racing and worried'
        },
        {
            'name': 'Stress',
            'mood': 4,
            'habits': 0.3,
            'text': 'so stressed with deadlines, feeling overwhelmed and burned out'
        },
        {
            'name': 'Healthy',
            'mood': 8,
            'habits': 0.9,
            'text': 'feeling great today, accomplished my goals and energetic'
        }
    ]
    
    print("🧪 TESTING ALL CONDITIONS")
    print("=" * 50)
    
    for test in test_cases:
        print(f"\n📋 {test['name'].upper()}")
        print("-" * 30)
        result = predictor.predict_condition(test['mood'], test['habits'], test['text'])
        print(f"Predicted: {result['condition']}")
        print(f"Crisis intervention: {result['needs_crisis_intervention']}")
        print(f"First action: {result['resources']['immediate_actions'][0]}")

# HOW TO USE IN YOUR APP:
# 
# 1. Create predictor instance:
#    predictor = SimpleMentalHealthPredictor()
#
# 2. Get user inputs from your app (mood, habits, text)
#
# 3. Make prediction:
#    result = predictor.predict_condition(mood_score, habit_completion_rate, text_input)
#
# 4. Use the result in your app:
#    - result['condition'] -> what condition was detected
#    - result['needs_crisis_intervention'] -> True/False for crisis situations
#    - result['resources']['immediate_actions'] -> list of immediate actions
#    - result['resources']['coping_strategies'] -> list of coping strategies
#    - result['explanation'] -> simple explanation of why this was predicted

# QUICK TEST EXAMPLES:
# quick_assessment(2, 0.1, "feeling hopeless and want to give up")
# quick_assessment(8, 0.9, "great day feeling amazing and accomplished")
# test_all_conditions()

In [4]:
# TESTING CODE FOR YOUR MENTAL HEALTH PREDICTOR
# Copy the SimpleMentalHealthPredictor class first, then run this

# Test 1: Quick individual tests
def test_individual_cases():
    """Test individual scenarios one by one"""
    print("🔍 INDIVIDUAL TEST CASES")
    print("=" * 40)
    
    # Test suicidal ideation
    print("\n1️⃣ TESTING SUICIDAL IDEATION:")
    quick_assessment(1, 0.1, "I want to end it all, can't go on anymore")
    
    # Test depression
    print("\n2️⃣ TESTING DEPRESSION:")
    quick_assessment(3, 0.2, "feeling hopeless and worthless, everything seems meaningless")
    
    # Test anxiety  
    print("\n3️⃣ TESTING ANXIETY:")
    quick_assessment(5, 0.6, "really anxious about everything, heart racing and worried")
    
    # Test stress
    print("\n4️⃣ TESTING STRESS:")
    quick_assessment(4, 0.3, "so stressed with deadlines, feeling overwhelmed and burned out")
    
    # Test healthy state
    print("\n5️⃣ TESTING HEALTHY STATE:")
    quick_assessment(8, 0.9, "feeling great today, accomplished my goals and very energetic")

# Test 2: Edge cases
def test_edge_cases():
    """Test tricky or borderline cases"""
    print("\n\n🎯 EDGE CASE TESTING")
    print("=" * 40)
    
    # Mixed signals - good mood but bad habits
    print("\n🔄 MIXED SIGNALS - Good mood, bad habits:")
    quick_assessment(7, 0.1, "feeling okay today but haven't been taking care of myself")
    
    # No text input
    print("\n📝 NO TEXT INPUT:")
    quick_assessment(4, 0.5, "")
    
    # Mild symptoms
    print("\n😐 MILD SYMPTOMS:")
    quick_assessment(5, 0.5, "just feeling a bit off today, nothing major")

# Test 3: Your own custom tests
def test_custom_scenarios():
    """Add your own test scenarios here"""
    print("\n\n🛠️ CUSTOM TEST SCENARIOS")
    print("=" * 40)
    
    # Add your own tests here - examples:
    
    print("\n💼 WORK STRESS:")
    quick_assessment(3, 0.4, "deadline tomorrow and I'm panicking, so much pressure")
    
    print("\n🏠 FAMILY ISSUES:")
    quick_assessment(4, 0.3, "family problems making me feel sad and drained")
    
    print("\n🎉 REALLY GOOD DAY:")
    quick_assessment(9, 0.8, "amazing day, got promoted and feeling fantastic!")

# Test 4: Raw output testing (for integration)
def test_raw_outputs():
    """Test raw outputs to see what your app will receive"""
    print("\n\n⚙️ RAW OUTPUT TESTING (for app integration)")
    print("=" * 50)
    
    predictor = SimpleMentalHealthPredictor()
    
    # Test case
    result = predictor.predict_condition(3, 0.2, "feeling depressed and hopeless")
    
    print("Raw result dictionary:")
    print(f"- condition: '{result['condition']}'")
    print(f"- needs_crisis_intervention: {result['needs_crisis_intervention']}")
    print(f"- explanation: '{result['explanation']}'")
    print(f"- immediate_actions count: {len(result['resources']['immediate_actions'])}")
    print(f"- first immediate action: '{result['resources']['immediate_actions'][0]}'")
    
    return result

# Test 5: Batch testing with different inputs
def test_batch_scenarios():
    """Test multiple scenarios quickly"""
    print("\n\n📊 BATCH TESTING")
    print("=" * 40)
    
    predictor = SimpleMentalHealthPredictor()
    
    test_scenarios = [
        (2, 0.1, "want to kill myself", "Crisis"),
        (7, 0.8, "great day today", "Good day"),
        (3, 0.3, "so tired and hopeless", "Depression"),
        (6, 0.7, "worried about exam tomorrow", "Anxiety"),
        (4, 0.2, "too much work, overwhelmed", "Stress"),
        (5, 0.5, "just an average day", "Neutral")
    ]
    
    for mood, habits, text, label in test_scenarios:
        result = predictor.predict_condition(mood, habits, text)
        print(f"{label:12} -> {result['condition']:15} (Crisis: {result['needs_crisis_intervention']})")

# MAIN TESTING FUNCTION - RUN THIS!
def run_all_tests():
    """Run all tests to validate your model"""
    print("🧠 MENTAL HEALTH PREDICTOR - FULL TEST SUITE")
    print("=" * 60)
    
    # Run individual tests
    test_individual_cases()
    
    # Run edge cases  
    test_edge_cases()
    
    # Run custom scenarios
    test_custom_scenarios()
    
    # Run raw output test
    raw_result = test_raw_outputs()
    
    # Run batch test
    test_batch_scenarios()
    
    # Run the built-in comprehensive test
    print("\n\n🔄 RUNNING BUILT-IN COMPREHENSIVE TEST:")
    test_all_conditions()
    
    print("\n\n✅ ALL TESTS COMPLETED!")
    print("Your model is ready for integration into your app!")
    
    return "Testing complete"

# SIMPLE TESTING COMMANDS:
# 
# To test everything at once:
# run_all_tests()
#
# To test just one scenario:
# quick_assessment(mood_score, habit_completion_rate, "your text here")
#
# To test all main conditions:
# test_all_conditions()
#
# To see raw output for app integration:
# test_raw_outputs()

if __name__ == "__main__":
    # Uncomment to run when script is executed
    # run_all_tests()
    pass

In [5]:
run_all_tests()

🧠 MENTAL HEALTH PREDICTOR - FULL TEST SUITE
🔍 INDIVIDUAL TEST CASES

1️⃣ TESTING SUICIDAL IDEATION:
🧠 CONDITION: SUICIDAL_IDEATION
📝 EXPLANATION: Concerning language detected. Low habit completion suggests difficulty with self-care
🆘 CRISIS INTERVENTION NEEDED: True

💡 IMMEDIATE ACTIONS:
   • 🚨 If in immediate danger, call 911
   • 📞 National Suicide Prevention Lifeline: 988
   • 💬 Crisis Text Line: Text HOME to 741741

🔧 COPING STRATEGIES:
   • Reach out to a trusted person immediately
   • Remove harmful objects from your area
   • Try box breathing: 4 counts in, hold 4, out 4, hold 4

2️⃣ TESTING DEPRESSION:
🧠 CONDITION: DEPRESSION
📝 EXPLANATION: Low mood score (3/10). Low habit completion suggests difficulty with self-care. Language suggests depressive symptoms
🆘 CRISIS INTERVENTION NEEDED: False

💡 IMMEDIATE ACTIONS:
   • Take a 5-minute walk outside
   • Write down 3 things you're grateful for
   • Call or text a friend or family member

🔧 COPING STRATEGIES:
   • Try to maintain 

'Testing complete'